In [1]:
# === SparkSession Setup (Foco em Delta Lake - Sem Conflito com Iceberg) ===
import shutil
import os
from pyspark.sql import SparkSession

# Cria SparkSession OTIMIZADA para Delta (sem Iceberg para evitar conflitos estáticos)
spark = SparkSession.builder \
    .appName("DeltaLake_ETL_IPS") \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.0") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .enableHiveSupport() \
    .getOrCreate()

# Log level logo após criação (reduz spam)
spark.sparkContext.setLogLevel("WARN")

# Verificação IMEDIATA: Confirma suporte Delta (antes de qualquer config extra)
try:
    spark.sql("CREATE TABLE IF NOT EXISTS spark_catalog.default.test_delta (id INT) USING DELTA")
    spark.sql("DROP TABLE spark_catalog.default.test_delta")
    print("✅ Suporte a Delta Lake confirmado! (JAR carregado e catálogo OK).")
except Exception as e:
    print(f"❌ Erro no Delta: {e} - Verifique instalação do JAR ou reinicie o kernel.")

print("SparkSession criada com sucesso! Versão:", spark.version)
print("Catálogos disponíveis:", [c.name for c in spark.catalog.listCatalogs()])

# Opcional: Se quiser adicionar configs Iceberg DINÂMICAS (não estáticas), faça aqui - mas sem extensions/catalog
# Ex: spark.conf.set("spark.sql.catalog.iceberg_catalog.warehouse", "/tmp/iceberg")  # Só se não conflitar
# Para Iceberg full, reinicie e use config no builder (veja abaixo).

25/09/28 14:44:11 WARN Utils: Your hostname, DESKTOP-1P6TETU resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/09/28 14:44:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/nice_correia/.cache/pypoetry/virtualenvs/trabalho-spark-RjY8yXlH-py3.12/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nice_correia/.ivy2/cache
The jars for the packages stored in: /home/nice_correia/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-24289162-aa64-42f6-abfe-6d42e2f9616a;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 342ms :: artifacts dl 13ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |

✅ Suporte a Delta Lake confirmado! (JAR carregado e catálogo OK).
SparkSession criada com sucesso! Versão: 3.5.7
Catálogos disponíveis: ['spark_catalog']


In [2]:
# Cleanup rápido (rode se der erro)
spark.sql("DROP TABLE IF EXISTS delta_ips")
shutil.rmtree("/tmp/delta", ignore_errors=True)
os.makedirs("/tmp/delta", exist_ok=True)
print("Delta ETL completo e limpo!")
   


Delta ETL completo e limpo!


In [3]:
# Detecta caminho do CSV (ajuste se necessário - assume notebooks/ no root do projeto)
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir) if "notebooks" in notebook_dir else notebook_dir
raw_path = os.path.join(project_root, "data", "raw", "ips_brasil.csv")

print(f"Caminho CSV: {raw_path} (existe? {os.path.exists(raw_path)})")

if os.path.exists(raw_path):
    df = spark.read.option("header", "true").option("inferSchema", "true").csv(raw_path)
    print(f"DF carregado: {df.count()} linhas, {len(df.columns)} colunas.")
    df.select("`Código IBGE`", "`Município`", "`UF`", "`Índice de Progresso Social`").show(5, truncate=False)
else:
    print("❌ CSV não encontrado! Verifique path ou baixe ips_brasil.csv para data/raw/.")

Caminho CSV: /home/nice_correia/trabalho_spark/data/raw/ips_brasil.csv (existe? True)


DF carregado: 5570 linhas, 79 colunas.
+-----------+--------------------------+---+--------------------------+
|Código IBGE|Município                 |UF |Índice de Progresso Social|
+-----------+--------------------------+---+--------------------------+
|1100015    |Alta Floresta D'Oeste (RO)|RO |50.94710852687823         |
|1100023    |Ariquemes (RO)            |RO |55.97475391330499         |
|1100031    |Cabixi (RO)               |RO |51.36453973053614         |
|1100049    |Cacoal (RO)               |RO |61.84526595721548         |
|1100056    |Cerejeiras (RO)           |RO |58.70878800673873         |
+-----------+--------------------------+---+--------------------------+
only showing top 5 rows



In [4]:
# === Delta Lake: DDL (Criação da Tabela) ===
print("\n=== Delta Lake: DDL ===")
delta_path = "/tmp/delta/ips"

# Limpa tabela se existir (opcional, overwrite cuida disso)
spark.sql(f"DROP TABLE IF EXISTS delta_ips")

# Escreve o DataFrame como Delta com column mapping (para nomes especiais como 'Código IBGE')
df.write.format("delta").mode("overwrite").option("delta.columnMapping.mode", "name").save(delta_path)

# Registra como tabela gerenciada (habilita SQL queries)
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS delta_ips
    USING DELTA
    LOCATION '{delta_path}'
    TBLPROPERTIES ('delta.columnMapping.mode' = 'name')
""")

print("Tabela Delta criada com sucesso! (Column mapping habilitado para nomes especiais).")
print(f"Total de linhas na tabela: {spark.sql('SELECT COUNT(*) FROM delta_ips').collect()[0][0]}")

# Verificação rápida: Mostra as top 5 linhas originais
spark.sql("SELECT `Código IBGE`, `Município`, `UF`, `Índice de Progresso Social` FROM delta_ips ORDER BY `Índice de Progresso Social` DESC LIMIT 5").show(truncate=False)


=== Delta Lake: DDL ===


25/09/28 14:09:11 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/09/28 14:09:16 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`default`.`delta_ips` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


Tabela Delta criada com sucesso! (Column mapping habilitado para nomes especiais).


Total de linhas na tabela: 5570


+-----------+-------------------+---+--------------------------+
|Código IBGE|Município          |UF |Índice de Progresso Social|
+-----------+-------------------+---+--------------------------+
|3516853    |Gavião Peixoto (SP)|SP |74.49282395600983         |
|5300108    |Brasília (DF)      |DF |71.25189747327438         |
|3548906    |São Carlos (SP)    |SP |70.96059545595044         |
|5208707    |Goiânia (GO)       |GO |70.49282747376537         |
|3533601    |Nuporanga (SP)     |SP |70.4719550872065          |
+-----------+-------------------+---+--------------------------+



In [5]:
# === Delta Lake: INSERT ===
print("\n=== Delta Lake: INSERT ===")

# INSERT via SQL (só preenche colunas chave; o resto fica NULL ou default)
spark.sql("INSERT INTO delta_ips (`Código IBGE`, `Município`, `UF`, `Índice de Progresso Social`) VALUES ('9999999', 'Exemplo Fictício', 'XX', 99.0)")

print("INSERT: Linha fictícia adicionada com sucesso!")

# Verificação: Top 5 linhas ordenadas por IPS (deve incluir a nova no topo)
spark.sql("SELECT `Código IBGE`, `Município`, `UF`, `Índice de Progresso Social` FROM delta_ips ORDER BY `Índice de Progresso Social` DESC LIMIT 5").show(truncate=False)

# Conta total (deve ser +1)
print(f"Total de linhas após INSERT: {spark.sql('SELECT COUNT(*) FROM delta_ips').collect()[0][0]}")


=== Delta Lake: INSERT ===
INSERT: Linha fictícia adicionada com sucesso!


+-----------+-------------------+---+--------------------------+
|Código IBGE|Município          |UF |Índice de Progresso Social|
+-----------+-------------------+---+--------------------------+
|9999999    |Exemplo Fictício   |XX |99.0                      |
|3516853    |Gavião Peixoto (SP)|SP |74.49282395600983         |
|5300108    |Brasília (DF)      |DF |71.25189747327438         |
|3548906    |São Carlos (SP)    |SP |70.96059545595044         |
|5208707    |Goiânia (GO)       |GO |70.49282747376537         |
+-----------+-------------------+---+--------------------------+

Total de linhas após INSERT: 5571


In [6]:
# === Delta Lake: UPDATE ===
print("\n=== Delta Lake: UPDATE ===")

# UPDATE via SQL (aumenta IPS em +1 para o município específico)
spark.sql("UPDATE delta_ips SET `Índice de Progresso Social` = `Índice de Progresso Social` + 1 WHERE `Código IBGE` = '1100015'")

print("UPDATE: IPS aumentado em +1 para Alta Floresta D'Oeste (código 1100015)!")

# Verificação: Mostra a linha atualizada
spark.sql("SELECT `Código IBGE`, `Município`, `Índice de Progresso Social` FROM delta_ips WHERE `Código IBGE` = '1100015'").show(truncate=False)

# Opcional: Verifica se mudou (compara com valor original do df, se quiser)
original_ips = df.filter("`Código IBGE` = '1100015'").select("`Índice de Progresso Social`").collect()[0][0]
print(f"IPS original (do DF): {original_ips} | IPS atualizado (na tabela): {spark.sql('SELECT `Índice de Progresso Social` FROM delta_ips WHERE `Código IBGE` = \"1100015\"').collect()[0][0]}")


=== Delta Lake: UPDATE ===


UPDATE: IPS aumentado em +1 para Alta Floresta D'Oeste (código 1100015)!


+-----------+--------------------------+--------------------------+
|Código IBGE|Município                 |Índice de Progresso Social|
+-----------+--------------------------+--------------------------+
|1100015    |Alta Floresta D'Oeste (RO)|51.94710852687823         |
+-----------+--------------------------+--------------------------+



[Stage 53:===============================================>        (42 + 7) / 50]

IPS original (do DF): 50.94710852687823 | IPS atualizado (na tabela): 51.94710852687823


In [7]:
# === Delta Lake: Time Travel ===
print("\n=== Delta Lake: Time Travel ===")

# Lista versões disponíveis (opcional: para ver histórico)
print("Versões disponíveis:")
spark.sql("DESCRIBE HISTORY delta_ips").select("version", "timestamp", "operation").show(10, truncate=False)

# Time Travel: Recupera versão 0 (dados originais, antes de qualquer DML)
print("Time Travel: Versão inicial (versão 0 - antes de UPDATE/DELETE).")
spark.sql("SELECT `Código IBGE`, `Município`, `UF`, `Índice de Progresso Social` FROM delta_ips VERSION AS OF 0 WHERE `Código IBGE` = '1100015'").show(truncate=False)

# Compara com estado atual (deve mostrar IPS original vs. atualizado)
print("Estado ATUAL (após UPDATE):")
spark.sql("SELECT `Código IBGE`, `Município`, `Índice de Progresso Social` FROM delta_ips WHERE `Código IBGE` = '1100015'").show(truncate=False)


=== Delta Lake: Time Travel ===
Versões disponíveis:
+-------+-----------------------+---------+
|version|timestamp              |operation|
+-------+-----------------------+---------+
|2      |2025-09-28 14:10:22.891|UPDATE   |
|1      |2025-09-28 14:09:51.947|WRITE    |
|0      |2025-09-28 14:09:15.965|WRITE    |
+-------+-----------------------+---------+

Time Travel: Versão inicial (versão 0 - antes de UPDATE/DELETE).


+-----------+--------------------------+---+--------------------------+
|Código IBGE|Município                 |UF |Índice de Progresso Social|
+-----------+--------------------------+---+--------------------------+
|1100015    |Alta Floresta D'Oeste (RO)|RO |50.94710852687823         |
+-----------+--------------------------+---+--------------------------+

Estado ATUAL (após UPDATE):
+-----------+--------------------------+--------------------------+
|Código IBGE|Município                 |Índice de Progresso Social|
+-----------+--------------------------+--------------------------+
|1100015    |Alta Floresta D'Oeste (RO)|51.94710852687823         |
+-----------+--------------------------+--------------------------+



In [8]:
# === Delta Lake: DELETE ===
print("\n=== Delta Lake: DELETE ===")

# DELETE via SQL (remove a linha fictícia pelo código único)
spark.sql("DELETE FROM delta_ips WHERE `Código IBGE` = '9999999'")

print("DELETE: Linha fictícia removida com sucesso!")

# Verificação: Top 5 linhas (não deve mais incluir a fictícia)
spark.sql("SELECT `Código IBGE`, `Município`, `UF`, `Índice de Progresso Social` FROM delta_ips ORDER BY `Índice de Progresso Social` DESC LIMIT 5").show(truncate=False)

# Conta total (deve voltar ao original, -1 do INSERT)
print(f"Total de linhas após DELETE: {spark.sql('SELECT COUNT(*) FROM delta_ips').collect()[0][0]}")


=== Delta Lake: DELETE ===


DELETE: Linha fictícia removida com sucesso!


+-----------+-------------------+---+--------------------------+
|Código IBGE|Município          |UF |Índice de Progresso Social|
+-----------+-------------------+---+--------------------------+
|3516853    |Gavião Peixoto (SP)|SP |74.49282395600983         |
|5300108    |Brasília (DF)      |DF |71.25189747327438         |
|3548906    |São Carlos (SP)    |SP |70.96059545595044         |
|5208707    |Goiânia (GO)       |GO |70.49282747376537         |
|3533601    |Nuporanga (SP)     |SP |70.4719550872065          |
+-----------+-------------------+---+--------------------------+

Total de linhas após DELETE: 5570


In [10]:
# === Delta Lake: Time Travel (Completo: Antes/Depois de INSERT, UPDATE e DELETE) ===
print("\n=== Delta Lake: Time Travel ===")
table_name = "spark_catalog.default.delta_ips"  # Full name; mude para "delta_ips" se alias

# Lista histórico completo (versões disponíveis)
print("Histórico de Versões (operação, timestamp e params):")
history_df = spark.sql(f"DESCRIBE HISTORY {table_name}")
history_df.select("version", "timestamp", "operation", "operationParameters").show(truncate=False, n=10)
total_versions = history_df.count()
print(f"Total de versões: {total_versions} (esperado: 4 após DDL/INSERT/UPDATE/DELETE)")

if total_versions < 2:
    print("⚠️ Poucas versões - rode mais DMLs (INSERT/UPDATE/DELETE) para demo completa.")
else:
    # Extrai IDs de versões chave (baseado em histórico típico: v0=DDL, v1=INSERT, v2=UPDATE, v3=DELETE)
    versions = history_df.select("version").orderBy("version").collect()
    v0 = versions[0][0] if len(versions) > 0 else 0
    v1_insert = versions[1][0] if len(versions) > 1 else None  # Após INSERT
    v2_update = versions[2][0] if len(versions) > 2 else None  # Após UPDATE (antes DELETE)
    v3_delete = versions[3][0] if len(versions) > 3 else None  # Após DELETE (atual)

    # Função auxiliar para query em versão específica
    def query_version(version, limit=1):
        return spark.sql(f"""
            SELECT `Código IBGE`, `Município`, `UF`, `Índice de Progresso Social` 
            FROM {table_name} VERSION AS OF {version} 
            WHERE `Código IBGE` IN ('9999999', '1100015')  -- Fictícia + Alta Floresta
            ORDER BY `Índice de Progresso Social` DESC
        """)

    # 1. Versão Inicial (v0: Antes de qualquer DML - só dados originais)
    print(f"\n1. Versão INICIAL (v{v0} - após DDL, antes INSERT/UPDATE/DELETE):")
    initial_df = query_version(v0)
    initial_df.show(truncate=False)
    initial_count = spark.sql(f"SELECT COUNT(*) FROM {table_name} VERSION AS OF {v0}").collect()[0][0]
    print(f"   Linhas totais: {initial_count} (dados CSV originais)")
    print("   Observação: Sem fictícia (9999999) e IPS original de Alta Floresta (~60-70)")

    # 2. Após INSERT (v1: Com linha fictícia adicionada)
    if v1_insert:
        print(f"\n2. Após INSERT (v{v1_insert} - fictícia adicionada, antes UPDATE/DELETE):")
        insert_df = query_version(v1_insert)
        insert_df.show(truncate=False)
        insert_count = spark.sql(f"SELECT COUNT(*) FROM {table_name} VERSION AS OF {v1_insert}").collect()[0][0]
        print(f"   Linhas totais: {insert_count} (+1 da fictícia 9999999 no topo com IPS 99.9)")
    else:
        print("\n2. Pulando após INSERT (versão não encontrada)")

    # 3. Após UPDATE, Antes DELETE (v2: Fictícia ainda presente, mas Alta Floresta atualizada)
    if v2_update:
        print(f"\n3. Após UPDATE, ANTES DELETE (v{v2_update} - UPDATE aplicado, fictícia ainda OK):")
        update_df = query_version(v2_update)
        update_df.show(truncate=False)
        update_count = spark.sql(f"SELECT COUNT(*) FROM {table_name} VERSION AS OF {v2_update}").collect()[0][0]
        print(f"   Linhas totais: {update_count} (mesmo que após INSERT; só IPS de 1100015 = 70.5)")
        print("   Observação: Fictícia (9999999) ainda existe; Alta Floresta atualizada")
    else:
        print("\n3. Pulando após UPDATE (versão não encontrada)")

    # 4. Após DELETE (v3/Atual: Fictícia removida, UPDATE preservado)
    current_version = history_df.orderBy("version", ascending=False).collect()[0][0]  # Última versão
    print(f"\n4. Após DELETE (v{current_version} - atual, fictícia removida):")
    delete_df = query_version(current_version)
    delete_df.show(truncate=False)
    current_count = spark.sql(f"SELECT COUNT(*) FROM {table_name}").collect()[0][0]
    print(f"   Linhas totais: {current_count} (-1 da fictícia; volta ao original)")
    print("   Observação: Fictícia (9999999) sumiu; Alta Floresta mantém IPS 70.5 (UPDATE intacto)")

    # Comparação Geral: Diferenças de contagem
    if v1_insert and v3_delete:
        print(f"\nResumo de Mudanças:")
        print(f"   Inicial (v{v0}): {initial_count} linhas")
        print(f"   Após INSERT (v{v1_insert}): {insert_count} linhas (+1)")
        print(f"   Após UPDATE (v{v2_update}): {update_count} linhas (sem mudança de contagem)")
        print(f"   Após DELETE (v{current_version}): {current_count} linhas (-1, volta ao inicial)")

# Opcional: Rollback para versão antes do DELETE (ex: v2 - desfaz DELETE, fictícia volta)
# print("\nTeste Opcional: Rollback para antes do DELETE (cuidado - altera atual!)")
# if v2_update:
#     spark.sql(f"RESTORE TABLE {table_name} TO VERSION AS OF {v2_update}")
#     print(f"Rollback executado para v{v2_update}! Agora fictícia voltou - verifique COUNT(*): {spark.sql(f'SELECT COUNT(*) FROM {table_name}').collect()[0][0]}")
#     # Para reverter rollback: RESTORE TO VERSION AS OF {current_version}

print("\nTime Travel Delta completo! Histórico imutável - DELETE removeu fictícia, mas UPDATE persiste em versões futuras.")


=== Delta Lake: Time Travel ===
Histórico de Versões (operação, timestamp e params):
+-------+-----------------------+---------+-----------------------------------------------+
|version|timestamp              |operation|operationParameters                            |
+-------+-----------------------+---------+-----------------------------------------------+
|3      |2025-09-28 14:11:08.592|DELETE   |{predicate -> ["(Código IBGE#7695 = 9999999)"]}|
|2      |2025-09-28 14:10:22.891|UPDATE   |{predicate -> ["(Código IBGE#4686 = 1100015)"]}|
|1      |2025-09-28 14:09:51.947|WRITE    |{mode -> Append, partitionBy -> []}            |
|0      |2025-09-28 14:09:15.965|WRITE    |{mode -> Overwrite, partitionBy -> []}         |
+-------+-----------------------+---------+-----------------------------------------------+

Total de versões: 4 (esperado: 4 após DDL/INSERT/UPDATE/DELETE)

1. Versão INICIAL (v0 - após DDL, antes INSERT/UPDATE/DELETE):


+-----------+--------------------------+---+--------------------------+
|Código IBGE|Município                 |UF |Índice de Progresso Social|
+-----------+--------------------------+---+--------------------------+
|1100015    |Alta Floresta D'Oeste (RO)|RO |50.94710852687823         |
+-----------+--------------------------+---+--------------------------+



   Linhas totais: 5570 (dados CSV originais)
   Observação: Sem fictícia (9999999) e IPS original de Alta Floresta (~60-70)

2. Após INSERT (v1 - fictícia adicionada, antes UPDATE/DELETE):


+-----------+--------------------------+---+--------------------------+
|Código IBGE|Município                 |UF |Índice de Progresso Social|
+-----------+--------------------------+---+--------------------------+
|9999999    |Exemplo Fictício          |XX |99.0                      |
|1100015    |Alta Floresta D'Oeste (RO)|RO |50.94710852687823         |
+-----------+--------------------------+---+--------------------------+



   Linhas totais: 5571 (+1 da fictícia 9999999 no topo com IPS 99.9)

3. Após UPDATE, ANTES DELETE (v2 - UPDATE aplicado, fictícia ainda OK):


+-----------+--------------------------+---+--------------------------+
|Código IBGE|Município                 |UF |Índice de Progresso Social|
+-----------+--------------------------+---+--------------------------+
|9999999    |Exemplo Fictício          |XX |99.0                      |
|1100015    |Alta Floresta D'Oeste (RO)|RO |51.94710852687823         |
+-----------+--------------------------+---+--------------------------+



   Linhas totais: 5571 (mesmo que após INSERT; só IPS de 1100015 = 70.5)
   Observação: Fictícia (9999999) ainda existe; Alta Floresta atualizada

4. Após DELETE (v3 - atual, fictícia removida):


+-----------+--------------------------+---+--------------------------+
|Código IBGE|Município                 |UF |Índice de Progresso Social|
+-----------+--------------------------+---+--------------------------+
|1100015    |Alta Floresta D'Oeste (RO)|RO |51.94710852687823         |
+-----------+--------------------------+---+--------------------------+

   Linhas totais: 5570 (-1 da fictícia; volta ao original)
   Observação: Fictícia (9999999) sumiu; Alta Floresta mantém IPS 70.5 (UPDATE intacto)

Resumo de Mudanças:
   Inicial (v0): 5570 linhas
   Após INSERT (v1): 5571 linhas (+1)
   Após UPDATE (v2): 5571 linhas (sem mudança de contagem)
   Após DELETE (v3): 5570 linhas (-1, volta ao inicial)

Time Travel Delta completo! Histórico imutável - DELETE removeu fictícia, mas UPDATE persiste em versões futuras.
